#### Week 3 Data Science Assignment : Question 2
##### Load Libraries

In [26]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Web scrapping
from bs4 import BeautifulSoup
import urllib.request
import csv

print('Libraries imported.')

Libraries imported.


In [27]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
boroughs = pd.DataFrame(columns=column_names)
boroughs

,PostalCode,Borough,Neighborhood


In [28]:
# specify the url
urlpage =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
urlpage

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [29]:
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(urlpage)
# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')
#soup

In [30]:
brhtbl = soup.find('table', class_='wikitable sortable')
trs = brhtbl.find_all('tr')
ln = len(trs)
ln

290

In [31]:
for tr in trs:
    tds = tr.find_all('td')
    ltd = len(tds)
    if ltd == 0:
        continue
    pc = tds[0].getText().replace('\n','')
    br = tds[1].getText().replace('\n','')
    nh = tds[2].getText().replace('\n','')
    if 'Not assigned' in br:
        continue
    if 'Not assigned' in nh:
        nh = br
        print (br, nh)
    '''
    #print(pc,br,nh)
    print(br, boroughs['Borough'] == br)
    try:
        ind = boroughs[boroughs['Borough'] == br ].index
        ts = boroughs[boroughs['Borough'] == br ]['Neighborhood']
        boroughs.loc[boroughs['Borough'] == br, ['Neighborhood']] = ts[0]+','+nh
        print(br,ind,ts)
    except:
        boroughs=boroughs.append({'PostalCode':pc,'Borough':br, 'Neighborhood':nh}, ignore_index=True)
    '''
    boroughs=boroughs.append({'PostalCode':pc,'Borough':br, 'Neighborhood':nh}, ignore_index=True)
   
    #print(boroughs)

Queen's Park Queen's Park


In [32]:
#boroughs

In [33]:
boroughs_grouped = boroughs.groupby(['Borough', 'PostalCode'])['Neighborhood'].apply(','.join).reset_index()

In [34]:
boroughs_grouped.shape

(103, 3)

##### Get latitude longitude for each postal code

###### Let's try using Geocoder 

Install Geocoder

In [35]:
!pip install geocoder

###### Lets try for postal code: 'M6M'

In [36]:

import geocoder # import geocoder

postal_code = 'M6M'
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
print('{}, Toronto, Ontario'.format(postal_code))
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  #print(g)
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]


"\nimport geocoder # import geocoder\n\npostal_code = 'M6M'\n# initialize your variable to None\nlat_lng_coords = None\n\n# loop until you get the coordinates\nprint('{}, Toronto, Ontario'.format(postal_code))\nwhile(lat_lng_coords is None):\n  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n  #print(g)\n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\n"

###### Didn't return after a long time hence had to interrupt kernel
###### Let's use the file at http://cocl.us/Geospatial_data

###### Let's get the file

In [37]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')


Data downloaded!


###### Read the file

In [38]:
geocodedt = pd.read_csv('Geospatial_Coordinates.csv')
geocodedt.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geocodedt.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


###### Merge the boroughs and postal lat/long data

In [39]:
boroughs_latlng = pd.merge(boroughs_grouped, geocodedt, on='PostalCode')

In [40]:
boroughs_latlng.head(15)

,Borough,PostalCode,Neighborhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park,Summerhill East",43.689574,-79.383160
5,Central Toronto,M4V,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049
6,Central Toronto,M5N,Roselawn,43.711695,-79.416936
7,Central Toronto,M5P,"Forest Hill North,Forest Hill West",43.696948,-79.411307
8,Central Toronto,M5R,"The Annex,North Midtown,Yorkville",43.672710,-79.405678
9,Downtown Toronto,M4W,Rosedale,43.679563,-79.377529
